# Imports and initialisation

In [1]:
import sqlite3
import pandas as pd
import requests
import json
conn = sqlite3.connect('./database.sqlite')
cur = conn.cursor()

# Goal scored

In [2]:
def goal_scored(division,season):
    if division not in ['E0','D1','D2']:
        print('The division argument is invalid. It must either be: \n D1 for Bundesliga, \n D2 for Bundesliga 2 \n E0 for Premier League')
    elif season < 1993 or season > 2016:
        print('Sorry, we only have data for seasons between 1993 and 2016!')
    else:
        conn = sqlite3.connect('./database.sqlite')
        cur = conn.cursor()
        query = """
                SELECT Div,HomeTeam, SUM(FTHG)+SUM(FTAG) as Goal_scored 
                FROM Matches 
                WHERE Season = {} 
                GROUP BY HomeTeam 
                ORDER BY Div, Goal_scored desc, HomeTeam""".format(season)
        df = pd.DataFrame(cur.execute(query).fetchall())
        df.columns = [x[0] for x in cur.description]
        g_s = sum(df[df['Div']==division]['Goal_scored'])
        return '{} goals were scored in {} during the {} season!'.format(g_s, division, season)

In [326]:
goal_scored('D1',2011)

'875 goals were scored in D1 during the 2011 season!'

# DarkSky API

## Generate the dataset

In [299]:
conn = sqlite3.connect('./database.sqlite')
cur = conn.cursor()
query = """
        SELECT *,
            CASE FTR 
                WHEN 'A' THEN AwayTeam
                WHEN 'H' THEN HomeTeam
                ELSE Null
            END as 'Winner',
            CASE FTR 
                WHEN 'A' THEN HomeTeam
                WHEN 'H' THEN AwayTeam
                ELSE Null
            END as 'Loser'
        FROM Matches
        WHERE Season = 2011
        """
df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
df['Weather'] = pd.Series(0, index=df.index)
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner,Loser,Weather
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,Nurnberg,0
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,Stuttgart,0
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,Wolfsburg,0
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz,Bayern Munich,0
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A,Man United,Sunderland,0
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H,Swansea,Liverpool,0
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H,Tottenham,Fulham,0
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A,Arsenal,West Brom,0


### Use the API to populate the weather column

In [359]:
apikey = dict(json.load(open('/home/antoine/.apikeys')))['Darksky']


def dkSky(division, date):
    lat = None
    long = None
    if division == 'E0':
        lat = 51.528308
        lon = -0.3817815
    else:
        lat = 52.5065133
        lon = 13.1445703
    response = requests.get('https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.
    format(apikey,lat,lon,date))
    return(response.json()['daily']['data'][0]['icon'])


for date in df['Date'][:5]:
    df['Weather'].apply(lambda x: dkSky(df['Div'][x],df['Date'][x]))

# df



# for date in df['Date']:
# df['Weather'] = [requests.get(
#     'https://api.darksky.net/forecast/{}/51.528308,-0.3817815,{}T16:00:00'.
#     format(apikey,date)).
#                  json()['daily']['data'][0]['icon'] 
#                  if x == 'E0' else 0 for x in df['Div']]




# requests.get('https://api.darksky.net/forecast/{}/51.528308,-0.3817815,{}T16:00:00'.format(apikey,date)).json()['daily']['data'][0]['icon']


# apply(lambda  x: requests.get(
#     'https://api.darksky.net/forecast/{}/51.528308,-0.3817815,{}T16:00:00'.
#     format(apikey,date)).
#                  json()['daily']['data'][0]['icon']
#       if x




# for date in df['Date'][:5]:
#     while df[df['Div'] == 'E0':
#         print('OK')
        # London
#         lat = 51.528308
#         long = -0.3817815
#         url='https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(apikey,lat,long,date)
#         response = requests.get(url)
#         df['weather'] = response.json()['daily']['data'][0]['icon']
#     else:
        # Berlin
#         lat = 52.5065133
#         long = 13.1445703
#         url='https://api.darksky.net/forecast/{}/{},{},{}T16:00:00'.format(apikey,lat,long,date)
#         response = requests.get(url)
#         df['weather'] = response.json()['daily']['data'][0]['icon']


#         JOIN Teams_in_Matches ON Matches.Match_ID = Teams_in_Matches.Match_ID
#         JOIN Unique_Teams ON Teams_in_Matches.Unique_Team_ID = Unique_Teams.Unique_Team_ID
#         JOIN Teams ON Unique_Teams.TeamName = Teams.TeamName
#         WHERE Matches.Season = 2011

KeyboardInterrupt: 

In [351]:
df['Div'][2]

'D1'

In [354]:
dkSky(df['Div'][1],df['Date'][1])

'partly-cloudy-day'

# Wins vs Losses in 2011 season

In [183]:
query = """
        SELECT *
        FROM Matches
        WHERE HomeTeam LIKE 'Middle%'
        """
df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,38439,E0,1995,1995-08-26,Middlesbrough,Chelsea,2,0,H
1,38464,E0,1995,1995-09-12,Middlesbrough,Southampton,0,0,D
2,38471,E0,1995,1995-09-16,Middlesbrough,Coventry,2,1,H
3,38489,E0,1995,1995-09-30,Middlesbrough,Blackburn,2,0,H
4,38509,E0,1995,1995-10-21,Middlesbrough,QPR,1,0,H
...,...,...,...,...,...,...,...,...,...
261,46700,E0,2016,2017-04-08,Middlesbrough,Burnley,0,0,D
262,46717,E0,2016,2017-04-17,Middlesbrough,Arsenal,1,2,A
263,46727,E0,2016,2017-04-26,Middlesbrough,Sunderland,1,0,H
264,46736,E0,2016,2017-04-30,Middlesbrough,Man City,2,2,D


In [297]:
conn = sqlite3.connect('./database.sqlite')
cur = conn.cursor()
query = """
        SELECT *,
            CASE FTR 
                WHEN 'A' THEN AwayTeam
                WHEN 'H' THEN HomeTeam
                ELSE Null
            END as 'Winner',
            CASE FTR 
                WHEN 'A' THEN HomeTeam
                WHEN 'H' THEN AwayTeam
                ELSE Null
            END as 'Loser'
        FROM Matches
        WHERE Season = 2011
        """
df = pd.DataFrame(cur.execute(query).fetchall())
df.columns = [x[0] for x in cur.description]
df = df.dropna()
df

df.pivot_table(index=['teams_perf.Teams'],columns=['Winner', 'Loser'], values)



# teams_perf = pd.DataFrame(cur.execute("SELECT TeamName FROM Unique_Teams").fetchall())
# teams_perf.columns = ['Teams']

# matches_lost = df.pivot_table(index='Winner', values='Loser', aggfunc='count')
# matches_lost

# teams_perf = pd.DataFrame(cur.execute("SELECT TeamName FROM Unique_Teams").fetchall())
# teams_perf.columns = ['Teams']
# teams_perf['Wins'] = pd.Series(0, index=teams_perf.index)
# teams_perf['Losses'] = pd.Series(0, index=teams_perf.index)
# teams_perf
# wnl = df.pivot_table(index=teams_perf.Wins,values='Winner',aggfunc='count')
# wnl

# df['Winner'].where(df['Winner'] == team)
# for team in teams_perf['Teams']:

#         print('ddd')
#     else:
#         print('boom')
#         teams_perf['Wins'] += 1
# teams_perf
# query = """
#         SELECT *
#         FROM Unique_Teams
#         JOIN Teams_in_Matches ON Unique_Teams.Unique_Team_ID = Teams_in_Matches.Unique_Team_ID
#         JOIN Matches ON Teams_in_Matches.Match_ID = Matches.Match_ID
#         GROUP BY Matches.Div, Unique_Teams.TeamName
#         """
# teams = pd.DataFrame(cur.execute(query).fetchall())
# teams.columns = [x[0] for x in cur.description]
# teams[50:99]

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Winner,Loser
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,Nurnberg
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich,Stuttgart
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich,Wolfsburg
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz,Bayern Munich
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,M'gladbach,Bayern Munich
...,...,...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A,Man United,Sunderland
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H,Swansea,Liverpool
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H,Tottenham,Fulham
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A,Arsenal,West Brom


In [ ]:
weather

# MongoDB

In [ ]:
query = """
        SELECT TeamName
        FROM Unique_Teams
        """

inputMongoDB = pd.DataFrame(cur.execute(query).fetchall())
inputMongoDB.columns = [x[0] for x in cur.description]
inputMongoDB